In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.driver.memory", '4g')
    .config("spark.executor.memory", '8g')
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.parquet.enableVectorizedReader","false")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

In [3]:
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [4]:
sdf_consumer = spark.read.csv('../data/tables/tbl_consumer.csv', sep='|', header=True)
sdf_consumer.show(5, truncate=False)

+-----------------+-----------------------------+-----+--------+------+-----------+
|name             |address                      |state|postcode|gender|consumer_id|
+-----------------+-----------------------------+-----+--------+------+-----------+
|Yolanda Williams |413 Haney Gardens Apt. 742   |WA   |6935    |Female|1195503    |
|Mary Smith       |3764 Amber Oval              |NSW  |2782    |Female|179208     |
|Jill Jones MD    |40693 Henry Greens           |NT   |862     |Female|1194530    |
|Lindsay Jimenez  |00653 Davenport Crossroad    |NSW  |2780    |Female|154128     |
|Rebecca Blanchard|9271 Michael Manors Suite 651|WA   |6355    |Female|712975     |
+-----------------+-----------------------------+-----+--------+------+-----------+
only showing top 5 rows



Convert postcodes to strings with leading zeros

In [5]:
sdf_consumer = sdf_consumer.withColumn('postcode', F.format_string("%04d", F.col('postcode').cast('int')))

In [7]:
sdf_consumer.count()

499999

In [12]:
VIC_sdf = sdf_consumer.where(
    (F.col('state') == 'VIC' ))

NSW_sdf = sdf_consumer.where(
    (F.col('state') == 'NSW' ))

QLD_sdf = sdf_consumer.where(
    (F.col('state') == 'QLD' ))

WA_sdf = sdf_consumer.where(
    (F.col('state') == 'WA' ))

TAS_sdf = sdf_consumer.where(
    (F.col('state') == 'TAS' ))

SA_sdf = sdf_consumer.where(
    (F.col('state') == 'SA' ))

ACT_sdf = sdf_consumer.where(
    (F.col('state') == 'ACT' ))

NT_sdf = sdf_consumer.where(
    (F.col('state') == 'NT' ))

In [9]:
VIC_sdf= VIC_sdf.where(
    ((F.col('postcode') >=  '3000') & (F.col('postcode') <= '3999'))
    | ((F.col('postcode') >=  '8000') & (F.col('postcode') <= '8999'))
)

117376

In [15]:
NT_sdf= NT_sdf.where(
    (F.col('postcode') >=  '0800') & (F.col('postcode') <= '0999'))

In [ ]:
TAS_sdf= TAS_sdf.where(
    (F.col('postcode') >=  '7000') & (F.col('postcode') <= '7999'))

In [ ]:
WA_sdf= WA_sdf.where(
    ((F.col('postcode') >=  '6000') & (F.col('postcode') <= '6797'))
    | ((F.col('postcode') >=  '6800') & (F.col('postcode') <= '6999'))
)

In [ ]:
SA_sdf= SA_sdf.where(
    (F.col('postcode') >=  '5000') & (F.col('postcode') <= '5999'))

In [ ]:
QLD_sdf= QLD_sdf.where(
    ((F.col('postcode') >=  '4000') & (F.col('postcode') <= '4999'))
    | ((F.col('postcode') >=  '9000') & (F.col('postcode') <= '9999'))
)

In [ ]:
NSW_sdf= NSW_sdf.where(
    ((F.col('postcode') >=  '1000') & (F.col('postcode') <= '2599'))
    | ((F.col('postcode') >=  '2619') & (F.col('postcode') <= '2899'))
    | ((F.col('postcode') >=  '2921') & (F.col('postcode') <= '2999'))
)

In [ ]:
ACT_sdf= ACT_sdf.where(
    ((F.col('postcode') >=  '0200') & (F.col('postcode') <= '0300'))
    | ((F.col('postcode') >=  '2600') & (F.col('postcode') <= '2618'))
    | ((F.col('postcode') >=  '2900') & (F.col('postcode') <= '2920'))
)

In [ ]:
cleaned_consumer = VIC_sdf.union(NSW_sdf)
cleaned_consumer = cleaned_consumer.union(QLD_sdf)
cleaned_consumer = cleaned_consumer.union(WA_sdf)
cleaned_consumer = cleaned_consumer.union(TAS_sdf)
cleaned_consumer = cleaned_consumer.union(SA_sdf)
cleaned_consumer = cleaned_consumer.union(ACT_sdf)
cleaned_consumer = cleaned_consumer.union(NT_sdf)

cleaned_consumer.count()

In [ ]:
cleaned_consumer.write.mode('overwrite').parquet('../data/curated/cleaned_consumers.parquet')